In [48]:
%%configure -f
{"name": "remotesparkmagics-sue", "executorMemory": "12G", "numExecutors": 10, 
 "executorCores": 4,
 "conf": {"spark.jars": "/system/jar/simpleHTM.jar,/system/jar/jhealpix.jar"}}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
124,application_1580142637008_0133,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
124,application_1580142637008_0133,spark,idle,Link,Link,✔


In [49]:
import simple.HTMindex
import healpix.jhu.Healpix
import org.apache.spark.sql.{SparkSession, SaveMode, Row, DataFrame}
import org.apache.spark.sql.functions.udf
import java.util.Calendar

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import simple.HTMindex
import healpix.jhu.Healpix
import org.apache.spark.sql.{SparkSession, SaveMode, Row, DataFrame}
import org.apache.spark.sql.functions.udf
import java.util.Calendar


In [50]:
object HTMUtils extends Serializable {
    var htmindex= new HTMindex() with Serializable
    val htmid: (Double, Double) => Long = htmindex.lookupId(_,_)

    val htmidUDF=udf(htmid)
    
}

object HEALPixUtils extends Serializable {
    var hp = new Healpix() with Serializable
    val healpixid: (Double, Double) => Long = hp.ang2pix(_,_)
    val healpixidUDF=udf(healpixid)
    
}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

defined object HTMUtils
defined object HEALPixUtils


In [51]:
print("hello from scala")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

hello from scala

# can we get this login thing to work in scala?
this was a pain, it would be better if i ACTUALL knew scala i guess

In [52]:
import org.apache.spark.rdd.RDD

val authDF = spark.read.json("hdfs:///.config/creds.json")
val m = authDF.first.getValuesMap[Any](authDF.schema.fieldNames)

val username = (m("jdbc_username"))
//println(username)
val password = (m("jdbc_password"))
//println(password)




FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.rdd.RDD
authDF: org.apache.spark.sql.DataFrame = [jdbc_password: string, jdbc_username: string]
m: Map[String,Any] = Map(jdbc_password -> fooRiuzg54, jdbc_username -> admin)
username: Any = admin
password: Any = fooRiuzg54


# get data from parquet files into dataframe, add extra computed columns for htmid and healpixid

In [57]:
val datafile = "/user/hive/warehouse/object"
val objDF = spark.read.parquet(datafile)

val newDF = objDF.withColumn("htmid",HTMUtils.htmidUDF(objDF("ra"),objDF("decl"))).withColumn("healpixid", HEALPixUtils.healpixidUDF(objDF("ra"), objDF("decl")))

newDF.write.format("parquet").mode("overwrite").saveAsTable("object_newcols")




FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

datafile: String = /user/hive/warehouse/object
objDF: org.apache.spark.sql.DataFrame = [deepSourceId: bigint, ra: double ... 234 more fields]
newDF: org.apache.spark.sql.DataFrame = [deepSourceId: bigint, ra: double ... 236 more fields]


NameError: name 'println' is not defined